In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb

In [4]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#2. Get the file
data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

In [11]:
#2. Get the file
data_path     = '../data'
codes_path    = '../codes'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv')

In [12]:
train['mal'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['comment_text', 'mal'])

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)
test_labelled = test_labelled.drop_duplicates(subset=['comment_text'])

test_unlabelled.comment_text.fillna("empty", inplace=True)
test_unlabelled = test_unlabelled.drop_duplicates(subset=['comment_text'])

In [13]:
rs=42
def strat_split(strat=False):
  global rs
  if strat:
      from sklearn.model_selection import train_test_split      
      X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
      X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)
      X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
      y = np.concatenate((y_train1, y_train2))

      X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
      y_test = np.concatenate((y_test1, y_test2))
  else:
      X = np.concatenate((train.comment_text, test_labelled.comment_text))
      y = np.concatenate((train.mal, test_labelled.mal))
      from sklearn.utils import shuffle
      from sklearn.model_selection import train_test_split
      X = shuffle(X, random_state=rs)
      y = shuffle(y, random_state=rs)      
      X, X_test, y, y_test  = train_test_split(X, y, stratify=y, test_size=0.3, random_state=rs )
      
  return X, X_test, y, y_test    
X, X_test, y, y_test = strat_split(True)

In [ ]:
# Ridge

In [14]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 40000)

word_vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=40000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents='unicode',
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

In [ ]:
auc_pr = 0.093434375824395
auc_roc = 0.02343524395
c = 0
C_parameter = np.arange(0.1, 1, 0.1) 

# use best C

C_parameter = [1]

In [ ]:
for c_p in C_parameter:  
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = LogisticRegression(C=c_p, solver='sag')
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            if len(C_parameter)==1:
                # print performance
                print(f'---------------------------------------------')
                print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
                print(f'---------------------------------------------')
                print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1

      if len(C_parameter)!=1:
             print(f'PARAMETER C = {c_p}')

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 4)}, AUC ROC = {round(np.array(roc).mean(), 4)}')
      print(f'-----------------------------------------------')
      print(f'')

In [ ]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = LogisticRegression(C=1, solver='sag')
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

In [ ]:
# RF

In [14]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 40000)

word_vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=40000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents='unicode',
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

In [16]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1


# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 4)}, AUC ROC = {round(np.array(roc).mean(), 4)}')
print(f'-----------------------------------------------')
print(f'')

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.0min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  2.8min
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  4.2min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    3.1s finished


---------------------------------------------
FOLD 0: AUC PR-C = 0.8463, AUC ROC = 0.9595
---------------------------------------------



[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.3min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  2.9min
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  4.1min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    3.1s finished


---------------------------------------------
FOLD 1: AUC PR-C = 0.8541, AUC ROC = 0.9641
---------------------------------------------



[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    8.3s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.1min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  2.8min
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  3.9min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    3.2s finished


...

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.0min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  2.5min
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  3.6min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    3.2s finished


---------------------------------------------
FOLD 3: AUC PR-C = 0.7881, AUC ROC = 0.9608
---------------------------------------------



[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    6.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.0min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  2.5min
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  3.6min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    2.3s


---------------------------------------------
FOLD 4: AUC PR-C = 0.7394, AUC ROC = 0.9615
---------------------------------------------

-----------------------------------------------
CV average: AUC PR-C = 0.8152, AUC ROC = 0.9615
-----------------------------------------------



[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    3.4s finished


In [17]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    8.7s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.3min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  3.1min
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  4.5min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    7.9s


-----------------------------------------
TEST: AUC PR-C = 0.8144, AUC ROC = 0.9627
-----------------------------------------



[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   11.5s finished


In [18]:
# SVM

In [25]:
penalty = ['l2', 'l1']
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
max_iter = [1000, 10000, 15000]

# best parameters
penalty = ['l1']
alpha = [0.00001]
max_iter = [1000] 

for p in penalty:
  for a in alpha:
    for i in max_iter:
      auc = []
      roc = []
      c = 0
      # print performance
      print(f'-------------')
      print(f'penalty {p}')
      print(f'alpha {a}')
      print(f'max_iter {i}')
      print(f'-------------')
      
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=p, alpha=a, max_iter=i)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1


# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 4)}, AUC ROC = {round(np.array(roc).mean(), 4)}')
print(f'-----------------------------------------------')
print(f'')

-------------
penalty l1
alpha 1e-05
max_iter 1000
-------------
---------------------------------------------
FOLD 0: AUC PR-C = 0.8622, AUC ROC = 0.9638
---------------------------------------------

---------------------------------------------
FOLD 1: AUC PR-C = 0.8614, AUC ROC = 0.9623
---------------------------------------------

---------------------------------------------
FOLD 2: AUC PR-C = 0.8645, AUC ROC = 0.9633
---------------------------------------------

---------------------------------------------
FOLD 3: AUC PR-C = 0.8112, AUC ROC = 0.9626
---------------------------------------------

---------------------------------------------
FOLD 4: AUC PR-C = 0.7806, AUC ROC = 0.9626
---------------------------------------------

-----------------------------------------------
CV average: AUC PR-C = 0.836, AUC ROC = 0.963
-----------------------------------------------



In [27]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=penalty[0], alpha=alpha[0], max_iter=max_iter[0])
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

-----------------------------------------
TEST: AUC PR-C = 0.827, AUC ROC = 0.9618
-----------------------------------------



In [30]:
# lightgbm

In [41]:
# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         100,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          20,
    'random_state':      rs,
}

In [42]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = lgb.LGBMClassifier(**lgb_params) 
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 4)}, AUC ROC = {round(np.array(roc).mean(), 4)}')
print(f'-----------------------------------------------')
print(f'')

---------------------------------------------
FOLD 0: AUC PR-C = 0.8498, AUC ROC = 0.9587
---------------------------------------------

---------------------------------------------
FOLD 1: AUC PR-C = 0.8542, AUC ROC = 0.9589
---------------------------------------------

---------------------------------------------
FOLD 2: AUC PR-C = 0.8488, AUC ROC = 0.9569
---------------------------------------------

---------------------------------------------
FOLD 3: AUC PR-C = 0.7982, AUC ROC = 0.9564
---------------------------------------------

---------------------------------------------
FOLD 4: AUC PR-C = 0.7647, AUC ROC = 0.9575
---------------------------------------------

-----------------------------------------------
CV average: AUC PR-C = 0.8296, AUC ROC = 0.9603
-----------------------------------------------



In [43]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = lgb.LGBMClassifier(**lgb_params) 
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

-----------------------------------------
TEST: AUC PR-C = 0.8189, AUC ROC = 0.9582
-----------------------------------------

